In [ ]:
# default_exp data

# data
> Provides dataset for icevision records and classmaps

In [ ]:
#export
import datetime
from typing import Union, Optional, List
import os
import shutil
import json
from copy import deepcopy

import numpy as np
import pandas as pd
from PIL import Image
import panel as pn

from icevision.core.record import BaseRecord
import icevision.parsers as parsers
from icevision.data.data_splitter import RandomSplitter
from icevision.core.bbox import BBox
from icevision.core.class_map import ClassMap

from fastprogress import master_bar, progress_bar

from icevision_dashboards.plotting.utils import draw_record_with_bokeh
from icevision_dashboards.core.data import *

In [ ]:
#hide
pn.extension()

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_dir = icedata.fridge.load_data()
test_class_map = icedata.fridge.class_map()
test_parser = icedata.fridge.parser(test_data_dir, test_class_map)
test_train_records, test_valid_records = test_parser.parse()

In [ ]:
#export
class RecordDataframeParser(parsers.Parser, parsers.FilepathMixin, parsers.SizeMixin, parsers.LabelsMixin):
    def __init__(self, record_dataframe):
        self.record_dataframe = record_dataframe
        
    def __iter__(self):
        for group in self.record_dataframe.groupby("filepath"):
            yield group[1]
    
    def imageid(self, o):
        return o.iloc[0]["id"]
    
    def filepath(self, o):
        return o.iloc[0]["filepath"]
    
    def image_width_height(self, o):
        width, height = o.iloc[0]["width"], o.iloc[0]["height"]
        return (width, height)
    
    def labels(self, o):
        return [annot[1]["label_num"] for annot in o.iterrows()]

In [ ]:
#export
class BboxRecordDataframeParser(RecordDataframeParser, parsers.BBoxesMixin):
    def bboxes(self, o):
        return [BBox(annot[1]["bbox_xmin"], annot[1]["bbox_ymin"], annot[1]["bbox_xmax"], annot[1]["bbox_ymax"]) for annot in o.iterrows()]

In [ ]:
#export
class MaskRecordDataframeParser(BboxRecordDataframeParser, parsers.MaskRCNN):
    def iscrowds(self, o):
        raise NotImplementedError()

    def masks(self, o):
        raise NotImplementedError()

In [ ]:
#export
class RecordDataset(GenericDataset):
    def __init__(self, records: Union[List[BaseRecord], ObservableList, str], class_map=None, name=None, description=None):
        if isinstance(records, str):
            self.load_from_file(records)
        else:
            self.records = records if isinstance(records, ObservableList) else ObservableList(records)
            self.class_map = class_map
        super().__init__(self.records, name=name, description=description)
        self.records.register_callback(self.reset_infered_data)
    
    def __repr__(self):
        base_string = ""
        for col in self.stats_dataset.columns:
            base_string += str(col) + ": " + str(self.stats_dataset[col][0]) + " | "
        base_string = base_string[:-2]
        return base_string
        
    def __getitem__(self, index):
        return self.records[index]
        
    def __len__(self):
        return len(self.records)
    
    @property
    def num_images(self):
        return len(self)
        
    @classmethod
    def load_from_record_dataframe(cls, record_data_df: pd.DataFrame, class_map=None, name=None, description=None):
        records = cls.parse_df_to_records(record_data_df)
        if class_map is None:
            class_map = cls.create_class_map_from_record_df(record_data_df)
        return cls(records, class_map=class_map, name=name, description=description)

    @staticmethod
    def create_class_map_from_record_df(record_df):
        sorted_labels = record_df["label"].unique()[np.argsort(record_df["label_num"].unique())].tolist()
        sorted_label_nums = sorted(record_df["label_num"].unique())
        label_map = {key: value for key, value in zip(sorted_label_nums, sorted_labels)}
        return ClassMap([label_map[i] if i in label_map.keys() else "unknown_"+str(i) for i in range(max(sorted_label_nums))])
    
    @staticmethod
    def parse_df_to_records(record_data_df):
        raise NotImplementedError()
    
    def load_from_file(self, path):
        data = json.load(open(path))
        df = pd.DataFrame(data["data"])
        records = self.parse_df_to_records(df)
        self.records = ObservableList(records)
        self.class_map = ClassMap(data["class_map"])
        self._name = data["name"]
        self._description = data["description"]
    
    def save(self, save_path):
        if not os.path.isdir(save_path):
            os.makedirs(save_path, exist_ok=True)
        save_name = "dataset" if self.name == "" else self.name
        if not os.path.isfile(os.path.join(save_path, save_name+".json")):
            save_name = save_name+".json"
        else:
            counter = 1
            while True:
                save_name = save_name+"("+str(counter)+").json"
                if os.path.isfile(os.path.join(save_path, save_name)):
                    counter += 1
                else:
                    break
        
        class_map = self.class_map if self.class_map is not None else self.create_class_map_from_record_df(df)
        save_data = {"name": self.name, "description": self.description, "data": self.data.to_dict(), "class_map": class_map.id2class}
        
        json.dump(save_data, open(os.path.join(save_path, save_name), "w"), default=str)
        
    def get_image_by_index(self, index, width, height):
        return draw_record_with_bokeh(self[index], width=None, height=height, return_figure=True)
    
    @classmethod
    def create_new_from_mask(cls, cls_instance, mask):
        selection = cls_instance.data[mask]
        filepaths = np.unique(selection["filepath"]).tolist()
        new_records = [record for record in cls_instance.records if str(record["filepath"]) in filepaths]
        return cls(new_records)

In [ ]:
#export
class DataDescriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        """Aggregates stats from a list of records and returns a pandas dataframe with the aggregated stats. The creation time is not necessarily the real creation time. 
        This depends on the OS, for more information see: https://docs.python.org/3/library/os.html#os.stat_result."""
        data = []
        for index,record in enumerate(obj.records):
            for label, bbox in zip(record["labels"], record["bboxes"]):
                file_stats = record["filepath"].stat()
                bbox_widht = bbox.xmax - bbox.xmin
                bbox_height = bbox.ymax - bbox.ymin
                area = bbox_widht * bbox_height
                area_normalized = area / (record["width"] * record["height"])
                bbox_ratio = bbox_widht / bbox_height
                data.append(
                    {
                        "id": record["imageid"], "width": record["width"], "height": record["height"], "label": label, 
                        "bbox_xmin": bbox.xmin, "bbox_xmax": bbox.xmax, "bbox_ymin": bbox.ymin, "bbox_ymax": bbox.ymax, "area": area, 
                        "area_normalized": area_normalized, "bbox_ratio": bbox_ratio, "record_index": index, "bbox_width": bbox_widht, 
                        "bbox_height": bbox_height, "filepath": str(record["filepath"]), "creation_date": datetime.datetime.fromtimestamp(file_stats.st_ctime), 
                        "modification_date": datetime.datetime.fromtimestamp(file_stats.st_mtime), "num_annotations": len(record["bboxes"])
                    }
                )
        data = pd.DataFrame(data)
        data["label_num"] = data["label"]
        if obj.class_map is not None:
            data["label"] = data["label"].apply(obj.class_map.get_id)
        return data

In [ ]:
#export
class StatsDescriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        stats_dict = {}
        stats_dict["no_imgs"] = [obj.data["filepath"].nunique()]
        stats_dict["no_classes"] = [obj.data["label"].nunique()]
        stats_dict["classes"] = [list(obj.data["label"].unique())]
        stats_dict["area_min"] = [obj.data["area"].min()]
        stats_dict["area_max"] = [obj.data["area"].max()]
        stats_dict["num_annotations_min"] = [obj.data["num_annotations"].min()]
        stats_dict["num_annotations_max"] = [obj.data["num_annotations"].max()]
        stats_dict["name"] = [obj._name]
        stats_dict["description"] = [obj._description]
        return pd.DataFrame(stats_dict)

In [ ]:
#export
class ImageStatsDescriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        """Creates a dataframe containing stats about the images."""
        stats_dict = {}
        stats_dict["Num. imgs."] = obj.data["filepath"].nunique()
        stats_dict["Min Num. Objects"] = obj.data["num_annotations"].min()
        stats_dict["Max Num. Objects"] = obj.data["num_annotations"].max()
        stats_dict["Avg. Objects/Img"] = round(obj.data["num_annotations"].mean(),2)
        df = pd.DataFrame.from_dict(stats_dict, orient="index").T
        return df

In [ ]:
#export
class ClassStatsDescriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        """Creates a dataframe containing stats about the object classes."""
        stats_dict = {}
        label_group = obj.data.groupby("label")
        for label, group in label_group:
            label_stats = {}
            label_stats["imgs"] = group["filepath"].nunique()
            label_stats["objects"] = group.shape[0]
            label_stats["avg_objects_per_img"] = label_stats["objects"]/label_stats["imgs"]
            label_stats["frac_of_labels"] = round(label_stats["objects"]/obj.data.shape[0], 2)
            stats_dict[label] = label_stats
        df = pd.DataFrame(stats_dict).T
        df = df.rename_axis('Class').reset_index()
        return df

In [ ]:
#export
class GalleryStatsDescriptorBbox(DatasetDescriptor):
    def calculate_description(self, obj):
        """Creates a dataframe containing the data for a gallery."""
        df = obj.data[["id", "area", "num_annotations", "label", "bbox_ratio", "bbox_width", "bbox_height", "width", "height"]].drop_duplicates().reset_index(drop=True)
        return df

In [ ]:
#export
class BboxRecordDataset(RecordDataset):
    data = DataDescriptorBbox()
    gallery_data = GalleryStatsDescriptorBbox()
    stats_dataset = StatsDescriptorBbox()
    stats_image = ImageStatsDescriptorBbox()
    stats_class = ClassStatsDescriptorBbox()
    stats = StatsDescriptorBbox()
    
    def __init__(self, records: Union[List[BaseRecord], ObservableList, str], class_map=None, name=None, description=None):
        super().__init__(records, class_map, name, description)
        self.record_index_image_id_map = {str(record["filepath"]): index for index, record in enumerate(self.records)}
        self.data = None
        self.gallery_data = None
        self.stats_dataset = None
        self.stats_image = None
        self.stats_class = None
    
    @staticmethod
    def parse_df_to_records(record_data_df):
        return BboxRecordDataframeParser(record_data_df).parse(RandomSplitter([1]))[0]
    
    def get_image_by_image_id(self, image_id, width, height):
        index = self.record_index_image_id_map[image_id]
        return draw_record_with_bokeh(self[index], display_bbox=True, width=None, height=height, return_figure=True)

In [ ]:
test_record_dataset = BboxRecordDataset(test_valid_records, test_class_map)
assert isinstance(test_record_dataset.data, pd.DataFrame)
assert isinstance(test_record_dataset.stats_dataset, pd.DataFrame)
assert isinstance(test_record_dataset.stats_class, pd.DataFrame)
assert isinstance(test_record_dataset.stats_image, pd.DataFrame)

In [ ]:
mask = np.array([False]*test_record_dataset.data.shape[0])
mask[1] = True
masked_test_record = test_record_dataset.create_new_from_mask(test_record_dataset, mask)
assert len(masked_test_record) == 1

In [ ]:
# selection = cls_instance.data[mask]
# filepaths = np.unique(selection["filepath"])
# new_records = [record for record in cls_instance.records if record["filepath"] in filepaths]
# return cls(new_records)

In [ ]:
test_old_record_dataset_stats = test_record_dataset.stats_dataset
test_record_dataset.records.list = test_train_records
test_new_record_dataset_stats = test_record_dataset.stats_dataset
assert all(test_old_record_dataset_stats != test_new_record_dataset_stats)

In [ ]:
test_record_dataset.data

In [ ]:
test_regenerated_record_dataset = BboxRecordDataset.load_from_record_dataframe(test_record_dataset.data)
assert len(test_regenerated_record_dataset.records) == len(test_record_dataset.records)

In [ ]:
shutil.rmtree("dump_dir", ignore_errors=True)
os.mkdir("dump_dir")
test_record_dataset.name = ""
test_record_dataset.save("dump_dir")
test_record_dataset.save("dump_dir")
assert len(os.listdir("dump_dir")) == 2
assert os.path.isfile("dump_dir/dataset.json")
test_loaded_record_dataset = BboxRecordDataset("dump_dir/dataset.json")
assert test_record_dataset.data.sort_values("area").shape == test_loaded_record_dataset.data.sort_values("id").shape
shutil.rmtree("dump_dir")

In [ ]:
# # Imports
# from icevision.all import *
# import icedata

# # Load the Fridge dataset
# path = icedata.fridge.load_data()

# # Get the class_map, a utility that maps from number IDs to classs names
# class_map = icedata.fridge.class_map()

# # Fridge parser: provided out-of-the-box
# parser = icedata.fridge.parser(data_dir=path, class_map=class_map)
# train_records, valid_records = parser.parse()

# # shows images with corresponding labels and boxes
# # show_records(train_records[:6], ncols=3, class_map=class_map, show=True)

# # Define transforms - using Albumentations transforms out of the box
# train_tfms = tfms.A.Adapter(
#     [*tfms.A.aug_tfms(size=384, presize=512), tfms.A.Normalize()]
# )
# valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(384), tfms.A.Normalize()])
# # Create both training and validation datasets
# train_ds = Dataset(train_records, train_tfms)
# valid_ds = Dataset(valid_records, valid_tfms)

# # Create both training and validation dataloaders
# train_dl = faster_rcnn.train_dl(train_ds, batch_size=14, num_workers=4, shuffle=True)
# valid_dl = faster_rcnn.valid_dl(valid_ds, batch_size=14, num_workers=4, shuffle=False)

# # Create model
# model = faster_rcnn.model(num_classes=len(class_map))

# # Define metrics
# metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

# # Train using fastai2
# learn = faster_rcnn.fastai.learner(
#     dls=[train_dl, valid_dl], model=model, metrics=metrics
# )

# learn.fine_tune(20, lr=1e-4)


# new_train_ds = Dataset(train_records, valid_tfms)
# samples_plus_losses, preds, losses_stats = faster_rcnn.interp.plot_top_losses(model=model, dataset=new_train_ds, sort_by="loss_total", n_samples=4)

In [ ]:
# Imports
from icevision.all import *
import icedata

# Load the Pets dataset
path = icedata.pets.load_data()

# Get the class_map, a utility that maps from number IDs to classs names
class_map = icedata.pets.class_map()

# Pets parser: provided out-of-the-box
parser = icedata.pets.parser(data_dir=path, class_map=class_map)
train_records, valid_records = parser.parse()

# shows images with corresponding labels and boxes
# show_records(train_records[:6], ncols=3, class_map=class_map, show=True)

# Define transforms - using Albumentations transforms out of the box
train_tfms = tfms.A.Adapter(
    [*tfms.A.aug_tfms(size=384, presize=512), tfms.A.Normalize()]
)
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(384), tfms.A.Normalize()])
# Create both training and validation datasets
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

# Create both training and validation dataloaders
train_dl = faster_rcnn.train_dl(train_ds, batch_size=10, num_workers=4, shuffle=True)
valid_dl = faster_rcnn.valid_dl(valid_ds, batch_size=10, num_workers=4, shuffle=False)

# Create model
model = faster_rcnn.model(num_classes=len(class_map))

# Define metrics
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

# Train using fastai2
learn = faster_rcnn.fastai.learner(
    dls=[train_dl, valid_dl], model=model, metrics=metrics
)

learn.fine_tune(5, lr=1e-4)

In [ ]:
new_train_ds = Dataset(train_records, valid_tfms)
samples_plus_losses, preds, losses_stats = faster_rcnn.interp.plot_top_losses(model=model, dataset=new_train_ds, sort_by="loss_total", n_samples=4)

In [ ]:
#export
class MetricBaseDataObjectDetection(DatasetDescriptor):
    """Calculates TP, FP, FN at the given ious. Requires the Dataset to have DataDescriptorBboxWithPredictions as the descriptor Data"""
    def __init__(self, data_descriptor_name="base_data", image_identification_col="filepath", ious=None):
        self.image_identification_col = image_identification_col
        self.data_descriptor_name = data_descriptor_name
        if ious is None:
            self.ious = np.arange(0.5, 1, 0.05).round(2)
        else:
            self.ious = ious
        
    def calculate_description(self, obj):
        data_list = []
        iou_bar = master_bar(self.ious, total=len(self.ious))
        iou_bar.main_bar.comment = f"iou: {self.ious[0]}"
        for index,iou in enumerate(iou_bar):
            iou_bar.main_bar.comment = f"iou: {self.ious[min(index+1, len(self.ious)-1)]}"
            data = getattr(obj, self.data_descriptor_name)
            data = data.copy()
            data["statistic"] = "FP"
            data.loc[data["is_prediction"] == False, "statistic"] = "FN"
            data["iou"] = iou
            for image_path, image_data in progress_bar(data.groupby(self.image_identification_col), parent=iou_bar):
                predictions = image_data[image_data["is_prediction"] == True]
                annotations = image_data[image_data["is_prediction"] == False]
                for index, annotation in annotations.iterrows():
                    x_overlap = np.maximum(predictions["bbox_xmax"]*0, np.minimum(predictions["bbox_xmax"], annotation["bbox_xmax"]) - np.maximum(predictions["bbox_xmin"], annotation["bbox_xmin"]))
                    y_overlap = np.maximum(predictions["bbox_xmax"]*0, np.minimum(predictions["bbox_ymax"], annotation["bbox_ymax"]) - np.maximum(predictions["bbox_ymin"], annotation["bbox_ymin"]))
                    intersections = x_overlap * y_overlap
                    ious = intersections / (predictions["area"] + annotation["area"] - intersections)
                    if any(np.logical_and(ious >= iou, predictions["label"] == annotation["label"])):
                        index_position = np.argmax(ious)
                        data.loc[index, "statistic"] = None
                        data.loc[predictions.index[index_position], "statistic"] = "TP"
                        # remove the annotation to avoid double counting
                        predictions = predictions.drop(index=predictions.index[index_position])
            data_list.append(data)
        return pd.concat(data_list).reset_index(drop=True)

In [ ]:
#export
class PrecisionRecallMetricsObjectDetection(DatasetDescriptor):
    def __init__(self, ious=None):
        if ious is None:
            self.ious = np.arange(0.5, 1, 0.05).round(2)
        else:
            self.ious = ious
            
    def calculate_description(self, obj):
        pass

In [ ]:
#export
class ObjectDetectionResultsDataset(GenericDataset):
    
    def __init__(self, dataframe, name=None, description=None):
        super().__init__(dataframe, name, description)        

    def save(self, path):
        if not os.path.exists(os.path.join(*path.split("/")[:-1])):
            os.makedirs(os.path.join(*path.split("/")[:-1]))
        self.base_data.to_csv(path)
        
    def get_image_by_image_id(self, image_id, width=None, height=None):
        df_pred = self.base_data[(self.base_data["filepath"] == image_id) & (self.base_data["is_prediction"] == True)]
        df_gt = self.base_data[(self.base_data["filepath"] == image_id) & (self.base_data["is_prediction"] == False)]
        
        parser_pred = BboxRecordDataframeParser(df_pred)
        parser_gt = BboxRecordDataframeParser(df_gt)
        res_gt = parser_gt.parse(show_pbar=False, autofix=False)[1][0]
        res_pred = parser_pred.parse(show_pbar=False, autofix=False)[1]
        if len(res_pred) == 0:
            res_pred = deepcopy(res_gt)
            res_pred["labels"] = []
            res_pred["bboxes"] = []
        else:
            res_pred = res_pred[0]
        plot_gt = draw_record_with_bokeh(res_gt, display_bbox=True, return_figure=True, width=width, height=height)
        plot_pred = draw_record_with_bokeh(res_pred, display_bbox=True, return_figure=True, width=width, height=height)
        return pn.Row(pn.Column(pn.Row("<b>Ground Truth</b>",  align="center"), plot_gt), pn.Column(pn.Row("<b>Prediction</b>",  align="center"), plot_pred))
        
    @classmethod
    def load(cls, path):
        df = pd.read_csv(path)
        return cls(df, None, None)
        
    @classmethod
    def init_from_preds_and_sampels(cls, predictions, sampels_plus_losses, padded_along_shortest=True, class_map=None, name=None, description=None):
        """The input_records are required because they are the only information source with the image stats(width, etc.) for the image on the disk."""
        data = []
        for index, (prediction, sampel_plus_loss) in enumerate(zip(predictions, samples_plus_losses)):
            # TODO: At the moment only resize_and_pad or resize are handelt. Check if there are other edge cases that need to be included
            # The correction requires that the sampel_plus_loss has the scaled image sizes (not the padded ones or the original ones)
            # correct the width and height to the values of the original image
            img = Image.open(sampel_plus_loss["filepath"])
            width = img.size[0]
            height = img.size[1]
            # use bool to int for padded_along_shortest and int(sampel_plus_loss["width"] < sampel_plus_loss["height"]) to avoid if branches
            factor = max(width, height)/max(sampel_plus_loss["width"], sampel_plus_loss["height"])
            padding = max(sampel_plus_loss["width"], sampel_plus_loss["height"]) - min(sampel_plus_loss["width"], sampel_plus_loss["height"])
            # at the end /2 due to symmetric padding
            correct_x = lambda x: factor * (x - int(padded_along_shortest) * int(sampel_plus_loss["width"] < sampel_plus_loss["height"]) * padding/2)
            correct_y = lambda y: factor * (y - int(padded_along_shortest) * int(sampel_plus_loss["width"] > sampel_plus_loss["height"]) * padding/2)
            for label, bbox, score in zip(prediction["labels"], prediction["bboxes"], prediction["scores"]):
                xmin, xmax, ymin, ymax = correct_x(bbox.xmin), correct_x(bbox.xmax), correct_y(bbox.ymin), correct_y(bbox.ymax)
                file_stats = sampel_plus_loss["filepath"].stat()
                bbox_widht = xmax - xmin
                bbox_height = ymax - ymin
                area = bbox_widht * bbox_height
                area_normalized = area / (width * height)
                bbox_ratio = bbox_widht / bbox_height
                data.append(
                    {
                        "id": sampel_plus_loss["imageid"], "width": width, "height": height, "label": label,
                        "score": score, "bbox_xmin": xmin, "bbox_xmax": xmax, "bbox_ymin": ymin, "bbox_ymax": ymax, "area": area, 
                        "area_normalized": area_normalized, "bbox_ratio": bbox_ratio, "record_index": index, "bbox_width": bbox_widht, 
                        "bbox_height": bbox_height, "filepath": str(sampel_plus_loss["filepath"]), "filename": str(sampel_plus_loss["filepath"]).split("/")[-1], "creation_date": datetime.datetime.fromtimestamp(file_stats.st_ctime), 
                        "modification_date": datetime.datetime.fromtimestamp(file_stats.st_mtime), "num_annotations": len(prediction["bboxes"]), "is_prediction": True,
                        "loss_classifier": sampel_plus_loss["loss_classifier"], "loss_box_reg": sampel_plus_loss["loss_box_reg"], "loss_objectness": sampel_plus_loss["loss_objectness"],
                        "loss_rpn_box_reg": sampel_plus_loss["loss_rpn_box_reg"], "loss_total": sampel_plus_loss["loss_total"]
                    }
                )
            for label, bbox in zip(sampel_plus_loss["labels"], sampel_plus_loss["bboxes"]):
                xmin, xmax, ymin, ymax = correct_x(bbox.xmin), correct_x(bbox.xmax), correct_y(bbox.ymin), correct_y(bbox.ymax)
                file_stats = sampel_plus_loss["filepath"].stat()
                bbox_widht = xmax - xmin
                bbox_height = ymax - ymin
                area = bbox_widht * bbox_height
                area_normalized = area / (width * height)
                bbox_ratio = bbox_widht / bbox_height
                data.append(
                    {
                        "id": sampel_plus_loss["imageid"], "width": width, "height": height, "label": label,
                        "score": 1, "bbox_xmin": xmin, "bbox_xmax": xmax, "bbox_ymin": ymin, "bbox_ymax": ymax, "area": area, 
                        "area_normalized": area_normalized, "bbox_ratio": bbox_ratio, "record_index": index, "bbox_width": bbox_widht, 
                        "bbox_height": bbox_height, "filepath": str(sampel_plus_loss["filepath"]), "filename": str(sampel_plus_loss["filepath"]).split("/")[-1], "creation_date": datetime.datetime.fromtimestamp(file_stats.st_ctime), 
                        "modification_date": datetime.datetime.fromtimestamp(file_stats.st_mtime), "num_annotations": len(prediction["bboxes"]), "is_prediction": False,
                        "loss_classifier": sampel_plus_loss["loss_classifier"], "loss_box_reg": sampel_plus_loss["loss_box_reg"], "loss_objectness": sampel_plus_loss["loss_objectness"],
                        "loss_rpn_box_reg": sampel_plus_loss["loss_rpn_box_reg"], "loss_total": sampel_plus_loss["loss_total"]
                    }
                )
        data = pd.DataFrame(data)
        data["label_num"] = data["label"]
        if class_map is not None:
            data["label"] = data["label"].apply(class_map.get_id)
        
        return cls(data, name, description)

In [ ]:
# odrd = ObjectDetectionResultsDataset.init_from_preds_and_sampels(preds, samples_plus_losses, class_map=class_map)
# odrd.save("test_results/pets_train.dat")
odrd = ObjectDetectionResultsDataset.load("test_results/pets_train.dat")
# odrd.base_data

In [ ]:
# Imports
from icevision.all import *
import icedata

# Load the Pets dataset
path = icedata.pets.load_data()

# Get the class_map, a utility that maps from number IDs to classs names
class_map = icedata.pets.class_map()

# Pets parser: provided out-of-the-box
parser = icedata.pets.parser(data_dir=path, class_map=class_map)
train_records, valid_records = parser.parse()

In [ ]:
[i for i in train_records if i["filepath"] == "/home/frederik/.icevision/data/pets/images/basset_hound_182.jpg"]

In [ ]:
train_records[0]

In [ ]:
draw_record_with_bokeh(train_records[0], display_bbox=True)

In [ ]:
# for (metric_base_dataath, iou), data in progress_bar(odrd.metric_base_data.groupby(["filepath", "iou"]), comment="Calculating precision and recall"):
#     tp = (data["statistic"] == "TP").sum()
#     fp = (data["statistic"] == "FP").sum()
#     fn = (data["statistic"] == "FN").sum()
#     precision = tp/(tp+fp) if tp+fp > 0 else 0
#     recall = tp/(tp+fn) if tp+fn > 0 else 0
#     odrd.metric_base_data.loc[data.index,"img_tp"] = tp
#     odrd.metric_base_data.loc[data.index,"img_fp"] = fp
#     odrd.metric_base_data.loc[data.index,"img_fn"] = fn
#     odrd.metric_base_data.loc[data.index,"precision"] = precision
#     odrd.metric_base_data.loc[data.index,"recall"] = recall